In [1]:
import shutup
shutup.please()

import gym
import src.environments.continuous.stock_trading  

import numpy as np
import pandas as pd 

In [2]:
env = gym.make('StockTradingEnvironment-v0', use_technical_indicators= [
    "macd",
    "boll_ub",
    "boll_lb",
    "rsi_30",
    "cci_30",
    "dx_30",
    "close_30_sma",
    "close_60_sma",
])

Shape of DataFrame:  (22065, 8)
Shape of DataFrame:  (754, 8)



In [3]:
from src.agents.actor_critic.a2c import A2CAgent

In [4]:
def environment():
    env = gym.make('StockTradingEnvironment-v0',
                   use_technical_indicators= [
        "macd",
        "boll_ub",
        "boll_lb",
        "rsi_30",
        "cci_30",
        "dx_30",
        "close_30_sma",
        "close_60_sma",
    ])
    
    #env.success_threshold =0.05 # 5%
    return env


In [5]:
agent=A2CAgent(environment, epochs=1, actor_learning_rate=0.000025,critic_learning_rate=0.000025,policy="CNN")
#agent.load()
environment().success_threshold

0.04373389232745828

In [6]:
agent.hash

'55dab581a5a039fbdc07bf5921848c57'

In [ ]:
agent.env.mode = "train"
agent.learning_log.episodes = 0
agent.learn(
    timesteps=-1, 
    log_every=1000,
    success_threshold_lookback=1000,
    success_strict=True,
)

In [ ]:
agent.env.mode = "test"
scores = []
success = 0

for i in range(2):
    state = agent.env.reset(visualize=True)
    score = 0
    reward = 0
    done = False
    initial_portfolio = agent.env.portfolio_value
    step = 0
    while not done:
        agent.env.render()
        #state = np.expand_dims(state, axis=0)
        action, action_onehot, prediction = agent.act(state)
        # Retrieve new state, reward, and whether the state is terminal
        next_state, reward, done, _ = agent.env.step(action)
        #print(action, reward, agent.env.portfolio_value)
        # Memorize (state, action, reward) for training
        #self.buffer.remember(np.expand_dims(state, axis=0), action_onehot, reward)
        # Update current state
        if done :
            if step < agent.env.window_size-1:
                success +=1
        step+=1
        state = next_state
        score += reward
    
    #print(score,initial_portfolio, agent.env.portfolio_value)
    scores.append(score)
    
print(np.mean(scores))
print(min(scores))
print(max(scores))
print(success)